## library import

In [35]:
# library import
import os
import time
import numpy as np
import scipy as sp
import pandas as pd
import urllib
import time

from datetime import datetime

from urllib.request import urlopen
from bs4 import BeautifulSoup

from IPython.display import clear_output

## 주가종목 리스트 테이블 생성

In [2]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
1,HSD엔진,82740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
2,KG케미칼,1390,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제",1989-08-25,12월,김경묵,http://www.kgchem.co.kr,울산광역시
3,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,10060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


In [3]:
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명','종목코드']].rename(columns={'회사명':'name', '종목코드':'code'})
code_df.head()

,name,code
0,GS글로벌,001250
1,HSD엔진,082740
2,KG케미칼,001390
3,LG이노텍,011070
4,OCI,010060


## 일별 시세 crawling

In [4]:
for index, row in code_df.iterrows():
    print(index, row['name'], row['code'])

0 GS글로벌 001250
1 HSD엔진 082740
2 KG케미칼 001390
3 LG이노텍 011070
4 OCI 010060
5 SK네트웍스 001740
6 SK이노베이션 096770
7 STX 011810
8 WISCOM 024070
9 갤럭시아에스엠 011420
10 경동도시가스 267290
11 경동인베스트 012320
12 고려아연 010130
13 고려제강 002240
14 극동유화 014530
15 까뮤이앤씨 013700
16 남양유업 003920
17 노루페인트 090350
18 대림씨엔에스 004440
19 대영포장 014160
20 대원제약 003220
21 대유에이텍 002880
22 대한해운 005880
23 동남합성 023450
24 동아쏘시오홀딩스 000640
25 동양 001520
26 롯데손해보험 000400
27 롯데쇼핑 023530
28 마니커 027740
29 모두투어리츠 204210
30 무림페이퍼 009200
31 부국증권 001270
32 삼성SDI 006400
33 삼성바이오로직스 207940
34 삼성출판사 068290
35 삼성카드 029780
36 삼성화재해상보험 000810
37 삼양홀딩스 000070
38 삼영전자공업 005680
39 삼영화학공업 003720
40 상상인증권 001290
41 선도전기 007610
42 선진 136490
43 세원셀론텍 091090
44 신송홀딩스 006880
45 신풍제약 019170
46 신한알파리츠 293940
47 신한지주 055550
48 에넥스 011090
49 영화금속 012280
50 우진 105840
51 원림 005820
52 윌비스 008600
53 유나이티드 033270
54 유니온머티리얼 047400
55 이리츠코크렙 088260
56 인지컨트롤스 023800
57 일동제약 249420
58 제이에스코퍼레이션 194370
59 제일약품 271980
60 제일연마 001560
61 제일파마홀딩스 002620
62 제주항공 089590
63 조일알미늄 0

728 베뉴지 019010
729 보령메디앙스 014100
730 부방 014470
731 브리지텍 064480
732 비씨월드제약 200780
733 비즈니스온 138580
734 비피도 238200
735 삼진엘앤디 054090
736 삼천리자전거 024950
737 삼현철강 017480
738 서부T&D 006730
739 서산 079650
740 서플러스글로벌 140070
741 성우테크론 045300
742 셀루메드 049180
743 셀리버리 268600
744 소프트센 032680
745 솔본 035610
746 시공테크 020710
747 시노펙스 025320
748 신라젠 215600
749 신성델타테크 065350
750 신스타임즈 056000
751 쎄노텍 222420
752 쎄트렉아이 099320
753 쎌바이오텍 049960
754 씨엔플러스 115530
755 아비코전자 036010
756 아이오케이 078860
757 아진엑스텍 059120
758 아프리카TV 067160
759 에너전트 041590
760 에스앤씨엔진그룹 900080
761 에스에프에이 056190
762 에이디칩스 054630
763 에이스침대 003800
764 에이치시티 072990
765 에이프로젠 H&G 109960
766 에프엔에스테크 083500
767 엘디티 096870
768 엠에스오토텍 123040
769 오텍 067170
770 와이엔텍 067900
771 원익QnC 074600
772 원익머트리얼즈 104830
773 원익홀딩스 030530
774 원풍 008370
775 웹스 196700
776 웹젠 069080
777 웹케시 053580
778 위닉스 044340
779 위메이드 112040
780 윈팩 097800
781 윙입푸드 900340
782 유비쿼스 264450
783 유비쿼스홀딩스 078070
784 유진테크 084370
785 유티아이 179900
786 이건홀딩스 039020
787 이노메트리 302430
788 이랜텍 05

1359 파라텍 033540
1360 파마리서치프로덕트 214450
1361 파이오링크 170790
1362 파인텍 131760
1363 펩트론 087010
1364 평화정공 043370
1365 푸드나무 290720
1366 프리엠스 053160
1367 플랜티넷 075130
1368 하나머티리얼즈 166090
1369 하이텍팜 106190
1370 한국선재 025550
1371 한국정보공학 039740
1372 한국큐빅 021650
1373 한류타임즈 039670
1374 한양이엔지 045100
1375 한프 066110
1376 현성바이탈 204990
1377 현우산업 092300
1378 홈캐스트 064240
1379 화신정공 126640
1380 휴네시온 290270
1381 흥구석유 024060
1382 힘스 238490
1383 SGA임베디드 224880
1384 골프존데카 183410
1385 그린플러스 186230
1386 디피코 163430
1387 로보쓰리 238500
1388 명진홀딩스 267060
1389 미디어젠 279600
1390 비나텍 126340
1391 비엔디생활건강 215050
1392 비엔에프코퍼레이션 271780
1393 셀젠텍 258250
1394 소프트캠프 210610
1395 수젠텍 253840
1396 씨앗 103660
1397 아이엘사이언스 122050
1398 엘리비젼 276240
1399 엘에이티 311060
1400 유투바이오 221800
1401 이에스산업 241510
1402 제노텍 066830
1403 젠큐릭스 229000
1404 큐엠씨 136660
1405 포레스팅블록체인 227420
1406 피엔에이치테크 239890
1407 CJ CGV 079160
1408 DB 012030
1409 DB손해보험 005830
1410 F&F 007700
1411 GS리테일 007070
1412 JW생명과학 234080
1413 KC그린홀딩스 009440
1414 LG유플러스 032640
1415 LG전자 066

2154 서희건설 035890
2155 성광벤드 014620
2156 세동 053060
2157 세운메디칼 100700
2158 세중 039310
2159 셀바스AI 108860
2160 솔브레인 036830
2161 시스웍 269620
2162 신일제약 012790
2163 신진에스엠 138070
2164 썬텍 122800
2165 쏠리드 050890
2166 씨아이에스 222080
2167 씨케이에이치 900120
2168 씨티씨바이오 060590
2169 아리온 058220
2170 아세아텍 050860
2171 아이씨케이 068940
2172 아이앤씨 052860
2173 아이티센 124500
2174 아진산업 013310
2175 안랩 053800
2176 알서포트 131370
2177 알에스오토메이션 140670
2178 알톤스포츠 123750
2179 알파홀딩스 117670
2180 에스씨디 042110
2181 에스에너지 095910
2182 에스텍 069510
2183 에스폴리텍 050760
2184 에이비엘바이오 298380
2185 에치에프알 230240
2186 엔시트론 101400
2187 엘앤에프 066970
2188 영풍정밀 036560
2189 예스티 122640
2190 오상자이엘 053980
2191 오파스넷 173130
2192 올릭스 226950
2193 옴니시스템 057540
2194 옵토팩 123010
2195 와이솔 122990
2196 와이오엠 066430
2197 와이제이엠게임즈 193250
2198 와토스코리아 079000
2199 우원개발 046940
2200 위즈코프 038620
2201 유니트론텍 142210
2202 유비케어 032620
2203 유안타제3호스팩 287410
2204 유안타제4호스팩 313750
2205 이미지스 115610
2206 이씨에스 067010
2207 이엘케이 094190
2208 이오테크닉스 039030
2209 이즈미디어 181340
2210 인터파크홀딩스 035080
221

In [110]:
stock_quotation = []

# test 용 주식종목 index 개수 지정 (max:2275)
index_num = 3000

# naver 주가 page 수 지정
crawling_page = 150

#크롤링 일자 지정
today = datetime.now().strftime('%Y.%m.%d')
start_dt = '2015.01.01'

start_time = time.time()
# 주가 종목에 대한 iteration
for index, row in code_df.iterrows():

    if index < index_num:
    
        # max page number 추출
        url = 'https://finance.naver.com/item/sise_day.nhn?code=' + row['code']
        html = urlopen(url)
        source = BeautifulSoup(html.read(), "lxml")

        maxPage = source.find_all("table", align="center")[0]

        if (maxPage.find_all("td", class_="pgRR")==[]):
            mpNum = 1
        else:
            maxPage = maxPage.find_all("td", class_="pgRR")[0]

            mpNum = int(maxPage.a.get("href").split('page=')[-1])
    
        # max page 각각 crawling
        for page in range(1, mpNum+1):
            # max page 개수 설정
            if page <= crawling_page:
                
                # crawling source 추출
                url = 'https://finance.naver.com/item/sise_day.nhn?code=' + row['code'] + '&page=' + str(page)
                html = urlopen(url)
                source = BeautifulSoup(html.read(), "lxml")
                srclist = source.find_all("tr")

                # source 내 date와 필요 정보만 추출
                for i in range(1, len(srclist)-1):
                    if ((srclist[i].span != None)&(start_dt <= srclist[i].td.text< today )):
                        quotation = [row['name'],                                       # 종목명
                                     row['code'],                                       # 종목코드
                                     pd.to_datetime(srclist[i].td.text),                # 일자    
                                     srclist[i].find_all("td", class_="num")[0].text,   # 종가
                                     srclist[i].find_all("td", class_="num")[2].text,   # 시가
                                     srclist[i].find_all("td", class_="num")[3].text,   # 고가
                                     srclist[i].find_all("td", class_="num")[4].text,   # 저가
                                     srclist[i].find_all("td", class_="num")[5].text]   # 거래량

                        stock_quotation.append(quotation)
                        
                        clear_output(wait=True)
                        duration = time.time() - start_time
                        print("종목명: {}, 종목코드: {}, 일자: {}, page: {}, max page: {}, runtime: {:d}h {:d}m {:.2f}s".
                              format(row['name'], row['code'], srclist[i].td.text, page, mpNum,int(duration/3600), int(int(duration/60)%60), float(duration%60)))

            else:
                break
    else:
        break
        
stock_df = pd.DataFrame(stock_quotation, columns=['name','code','date','closing_price','open_price','high_price','low_price','volume'])

종목명: 갤럭시아에스엠, 종목코드: 011420, 일자: 2018.12.27, page: 10, max page: 574, runtime: 0h 0m 12.93s


## csv file로 저장

In [ ]:
# stock_df.to_csv('일별시세.csv', encoding='euc-kr', index=False)